In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting
import statsmodels.api as sm

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,13,'R')
session.load()

In [ ]:
drivers = session.drivers[:10]

In [ ]:
total_laps = 44
race_dict = {}
for driver_number in drivers:
    df = session.laps.pick_drivers(driver_number)
    inter_laps = df[df.Compound == 'INTERMEDIATE'].Compound.count()

    df_tel_wet = df.pick_laps([i for i in range(1,inter_laps+1)]).pick_fastest().get_car_data()
    throt_wet = (df_tel_wet.Throttle >= 95).sum()/df_tel_wet.shape[0]

    df_tel_dry = df.pick_laps([i for i in range(inter_laps+1,total_laps+1)]).pick_fastest().get_car_data()
    throt_dry = (df_tel_dry.Throttle >= 95).sum()/df_tel_dry.shape[0]

    race_dict[session.get_driver(driver_number).Abbreviation] = [inter_laps, total_laps - inter_laps, throt_wet, throt_dry]

In [ ]:
race_df = pd.DataFrame(race_dict).T.set_axis(['WetLaps','DryLaps','WetThrottle%','DryThrottle%'],axis=1)

In [ ]:
race_df.mean()

In [ ]:
sc_laps = 4
wet_laps = 12
dry_laps = 32
margin_laps = 5

sc_fuel_cons = 1
dry_fuel_cons = 2.4
wet_by_dry = 0.485/0.616
wet_fuel_cons = round(dry_fuel_cons * wet_by_dry,1)

In [ ]:
avg_fuel_cons = sc_laps * sc_fuel_cons + (wet_laps - sc_laps) * wet_fuel_cons + dry_laps * dry_fuel_cons
est_fuel_load = avg_fuel_cons + margin_laps * dry_fuel_cons
est_fuel_load

In [ ]:
drivers

In [ ]:
def tire_deg(driver_number,skip_age):
    df = session.laps.pick_drivers(driver_number)[['LapNumber','LapTime','Compound','TyreLife']].pick_quicklaps().copy()
    df['LapTime'] = df['LapTime'].dt.total_seconds()
    df['FC_LapTime'] = df['LapTime'] - round((44 - df['LapNumber'])*dry_fuel_cons*0.035,2)

    x = df.iloc[skip_age:].TyreLife
    x = sm.add_constant(x)
    y = df.iloc[skip_age:].FC_LapTime

    model = sm.OLS(y, x).fit()
    return df.iloc[skip_age:], model.params['TyreLife'], model.rsquared

In [ ]:
fig, ax = plt.subplots(2,5,figsize=(15,6),sharex=True,sharey=True)

dc = 0
i = 0
while i <= 1:
    j = 0
    while j <= 4:
        dc += 1
        driver_number = drivers[dc-1]
        df, td, r2 = tire_deg(driver_number,4)
        color = plotting.get_driver_color(identifier=session.get_driver(driver_number).Abbreviation,session=session)
        sns.regplot(x=df.TyreLife,y=df.FC_LapTime,ax=ax[i][j],color=color)
        ax[i][j].set_ylim(104,109)
        ax[i][j].set_xlabel('')
        ax[i][j].set_ylabel('')
        ax[i][j].text(18.5,108.5,session.get_driver(driver_number).Abbreviation,color=color,fontsize=14,fontweight='bold')
        ax[i][j].text(10.5,107.5,f'Tire Deg: {td:0.3f} s/lap\n    R-squared: {r2:0.2f}',fontsize=12)
        j+=1
    i+=1


plt.show()